## 예측 모델 선정 단계

In [ ]:
import pandas as pd
# import numpy as np

df = pd.read_csv("../data/merged/traffic_weather_features.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.floor('D')


df = df.sort_values(["intersection", "datetime"])

# 1. Lag Features 생성
df["lag1"] = df.groupby("intersection")["traffic_volume"].shift(1)
df["lag2"] = df.groupby("intersection")["traffic_volume"].shift(2)
df["lag3"] = df.groupby("intersection")["traffic_volume"].shift(3)

# Rolling Features
df["roll3"] = df.groupby("intersection")["traffic_volume"].rolling(3).mean().reset_index(0,drop=True)
df["roll6"] = df.groupby("intersection")["traffic_volume"].rolling(6).mean().reset_index(0,drop=True)

df = df.dropna()  # lag 때문에 생긴 NaN 제거


# 2. train/test data 분리
test_size_days = 30 # 최근 30일을 test로 사용
split_date = df["date"].max() - pd.Timedelta(days=test_size_days)

train = df[df["date"] <= split_date]
test  = df[df["date"] > split_date]

drop_cols = ["traffic_volume", "intersection", "datetime", "date", "slot"]  # 문자열 컬럼 제거

X_train = train.drop(columns=drop_cols)
y_train = train["traffic_volume"]

X_test = test.drop(columns=drop_cols)
y_test = test["traffic_volume"]


# 3. 후보 1) 베이스라인 모델(LR)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

lr = LinearRegression()
lr.fit(X_train, y_train)

pred_lr = lr.predict(X_test)

mse_lr = mean_squared_error(y_test, pred_lr)
rmse_lr = mse_lr ** 0.5
mae_lr = mean_absolute_error(y_test, pred_lr)

print("Baseline (LR) RMSE:", rmse_lr)
print("Baseline (LR) MAE:", mae_lr)


# 4. 후보 2) RandomForest로 예측
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=12,
    random_state=42
)

rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, pred_rf)
rmse_rf = mse_rf ** 0.5
mae_rf = mean_absolute_error(y_test, pred_rf)

print("RF RMSE:", rmse_rf)
print("RF MAE:", mae_rf)


# 5. 후보 3) XGBoost 모델
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)

mse_xgb = mean_squared_error(y_test, pred_xgb)
rmse_xgb = mse_xgb ** 0.5
mae_xgb = mean_absolute_error(y_test, pred_xgb)

print("XGB RMSE:", rmse_xgb)
print("XGB MAE:", mae_xgb)




Baseline (LR) RMSE: 1.2985833714843464e-11
Baseline (LR) MAE: 9.540694462216198e-12
RF RMSE: 95.77556007962038
RF MAE: 58.96613497312451
XGB RMSE: 60.76022692338509
XGB MAE: 36.11786651611328


### 테스트 기간(7, 30, 60, 90일)별 모델 성능 비교

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# -----------------------------------
# 데이터 로드
# -----------------------------------
df = pd.read_csv("../data/merged/traffic_weather_features.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.floor('D')

target = "2공단3"
df = df[df["intersection"] == target].copy()

df = df.sort_values("datetime").reset_index(drop=True)

df['lag1'] = df.groupby("intersection")['traffic_volume'].shift(1)
df['lag2'] = df.groupby("intersection")['traffic_volume'].shift(2)
df['lag3'] = df.groupby("intersection")['traffic_volume'].shift(3)

df['roll3'] = df.groupby("intersection")['traffic_volume'].rolling(3).mean().reset_index(level=0, drop=True)
df['roll6'] = df.groupby("intersection")['traffic_volume'].rolling(6).mean().reset_index(level=0, drop=True)

df = df.dropna()

# -----------------------------------
# 비교할 테스트 기간 목록
# -----------------------------------
test_days_list = [7, 30, 60, 90]

results = []

drop_cols = ["traffic_volume", "intersection", "datetime", "date", "slot"]

for test_days in test_days_list:
    split_date = df["date"].max() - pd.Timedelta(days=test_days)

    train = df[df["date"] <= split_date]
    test = df[df["date"] > split_date]

    X_train = train.drop(columns=drop_cols)
    y_train = train["traffic_volume"]

    X_test = test.drop(columns=drop_cols)
    y_test = test["traffic_volume"]

    # -------------------------------
    # 1) Linear Regression
    # -------------------------------
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    pred_lr = lr.predict(X_test)

    rmse_lr = (mean_squared_error(y_test, pred_lr)) ** 0.5
    mae_lr = mean_absolute_error(y_test, pred_lr)

    # -------------------------------
    # 2) Random Forest
    # -------------------------------
    rf = RandomForestRegressor(
        n_estimators=50, max_depth=12, random_state=42 # 테스트 환경상 50으로 낮춰서 진행
    )
    rf.fit(X_train, y_train)
    pred_rf = rf.predict(X_test)

    rmse_rf = (mean_squared_error(y_test, pred_rf)) ** 0.5
    mae_rf = mean_absolute_error(y_test, pred_rf)

    # -------------------------------
    # 3) XGBoost
    # -------------------------------
    xgb = XGBRegressor(
        n_estimators=100, # 테스트 환경상 100으로 낮춰서 진행
        max_depth=6,
        learning_rate=0.05,
        random_state=42,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="hist",
        enable_categorical=False
    )
    xgb.fit(X_train, y_train)
    pred_xgb = xgb.predict(X_test)

    rmse_xgb = (mean_squared_error(y_test, pred_xgb)) ** 0.5
    mae_xgb = mean_absolute_error(y_test, pred_xgb)

    # 결과 저장
    results.append({
        "test_days": test_days,
        "LR_RMSE": rmse_lr,
        "LR_MAE": mae_lr,
        "RF_RMSE": rmse_rf,
        "RF_MAE": mae_rf,
        "XGB_RMSE": rmse_xgb,
        "XGB_MAE": mae_xgb
    })

# -----------------------------------
# 결과 표 출력
# -----------------------------------
results_df = pd.DataFrame(results)
print(results_df)


   test_days       LR_RMSE        LR_MAE     RF_RMSE      RF_MAE    XGB_RMSE  \
0          7  1.751006e-12  1.361184e-12  224.100296   97.111670  180.762950   
1         30  1.477610e-12  1.247242e-12  166.857979   77.414643  141.804429   
2         60  1.926105e-11  1.706878e-11  211.395520  105.276260  191.409962   
3         90  1.510762e-11  1.336149e-11  208.174734  104.063433  184.863780   

      XGB_MAE  
0  100.565369  
1   83.152451  
2  107.258423  
3  103.808151  


### 테스트 결과, 현재 데이터 구조에서는 XGBoost가 적합하다고 판단 -> XGBoost로 고정

In [20]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import numpy as np

# ----------------------------------
# 1. 데이터 준비
# ----------------------------------
df = pd.read_csv("../data/merged/traffic_weather_features.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.floor("D")

df = df.sort_values(["intersection", "datetime"])

# Lag Features 생성
df["lag1"] = df.groupby("intersection")["traffic_volume"].shift(1)
df["lag2"] = df.groupby("intersection")["traffic_volume"].shift(2)
df["lag3"] = df.groupby("intersection")["traffic_volume"].shift(3)

# Rolling Features 생성
df["roll3"] = df.groupby("intersection")["traffic_volume"].rolling(3).mean().reset_index(0,drop=True)
df["roll6"] = df.groupby("intersection")["traffic_volume"].rolling(6).mean().reset_index(0,drop=True)

df = df.dropna()  # lag 때문에 생긴 NaN 제거

# train/test 분리
test_size_days = 30 # 테스트 기간을 비교해 본 결과, 최근 30일 예측에서 가장 좋은 성능을 기록
split_date = df["date"].max() - pd.Timedelta(days=test_size_days)

train = df[df["date"] <= split_date]
test  = df[df["date"] > split_date]

drop_cols = ["traffic_volume","intersection","datetime","date","slot"]
X_train = train.drop(columns=drop_cols)
y_train = train["traffic_volume"]

X_test = test.drop(columns=drop_cols)
y_test = test["traffic_volume"]


# # ----------------------------------
# # 2. XGBoost Parameter Space
# # ----------------------------------
# param_dist = {
#     "n_estimators": [200, 300, 500, 800],
#     "learning_rate": [0.01, 0.03, 0.05, 0.1],
#     "max_depth": [4, 6, 8, 10],
#     "subsample": [0.6, 0.8, 1.0],
#     "colsample_bytree": [0.6, 0.8, 1.0],
#     "gamma": [0, 1, 5],
#     "min_child_weight": [1, 3, 5]
# }

# ----------------------------------
# 3. TimeSeriesSplit 설정
# ----------------------------------
tscv = TimeSeriesSplit(n_splits=3)

# ----------------------------------
# 4. Random Search
# ----------------------------------
xgb = XGBRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

# random_search = RandomizedSearchCV(
#     estimator=xgb,
#     param_distributions=param_dist,
#     n_iter=20,              # 탐색 횟수 (늘리면 성능↑, 시간↑)
#     scoring="neg_mean_squared_error",
#     cv=tscv,
#     verbose=2,
#     n_jobs=-1
# )

# 시계열 교차검증
cv_scores = cross_val_score(
    xgb,
    X_train,
    y_train,
    cv=tscv,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

rmse_scores = (-cv_scores) ** 0.5

# random_search.fit(X_train, y_train)

# print("\n🎯 Best Parameters:")
# print(random_search.best_params_)

# # ----------------------------------
# # 5. 최종 모델 재학습
# # ----------------------------------
# best_model = random_search.best_estimator_

# best_model.fit(X_train, y_train)

# # ----------------------------------
# # 6. Test Set 평가
# # ----------------------------------
# pred = best_model.predict(X_test)

# mse = mean_squared_error(y_test, pred)
# rmse = mse ** 0.5
# mae = mean_absolute_error(y_test, pred)

# print("\n🔥 Optimized XGBoost 성능")
# print("RMSE:", rmse)
# print("MAE:", mae)

print("=== TimeSeriesSplit CV 결과 (Baseline XGB) ===")
for i, s in enumerate(rmse_scores, 1):
    print(f"Fold {i} RMSE: {s:.2f}")
print(f"▶ 평균 RMSE: {rmse_scores.mean():.2f}")
print(f"▶ 표준편차: {rmse_scores.std():.2f}")


=== TimeSeriesSplit CV 결과 (Baseline XGB) ===
Fold 1 RMSE: 377.05
Fold 2 RMSE: 54.08
Fold 3 RMSE: 65.44
▶ 평균 RMSE: 165.52
▶ 표준편차: 149.64


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

# ============================================================
# 1. 데이터 불러오기
# ============================================================
df = pd.read_csv("../data/merged/traffic_weather_features.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.floor('D')

# 클러스터링 결과 불러오기
features = pd.read_csv("../data/merged/cluster_features.csv")
cluster_map = features[['intersection', 'cluster']]

# Merge cluster info
df = df.merge(cluster_map, on='intersection', how='left')

print("데이터셋 구조:")
print(df.head())


# ============================================================
# 2. Feature Engineering
# ============================================================

df = df.sort_values(["intersection", "datetime"]).reset_index(drop=True)

# Lag Features
df['lag1'] = df.groupby("intersection")['traffic_volume'].shift(1)
df['lag2'] = df.groupby("intersection")['traffic_volume'].shift(2)
df['lag3'] = df.groupby("intersection")['traffic_volume'].shift(3)

# More Powerful Lags
df['lag24'] = df.groupby("intersection")['traffic_volume'].shift(24)
df['lag48'] = df.groupby("intersection")['traffic_volume'].shift(48)
df['lag72'] = df.groupby("intersection")['traffic_volume'].shift(72)

# Rolling windows
df['roll3'] = df.groupby("intersection")['traffic_volume'].rolling(3).mean().reset_index(level=0, drop=True)
df['roll6'] = df.groupby("intersection")['traffic_volume'].rolling(6).mean().reset_index(level=0, drop=True)
df['roll24'] = df.groupby("intersection")['traffic_volume'].rolling(24).mean().reset_index(level=0, drop=True)
df['roll48'] = df.groupby("intersection")['traffic_volume'].rolling(48).mean().reset_index(level=0, drop=True)

# Cyclical Encoding (hour, dayofweek)
df['hour'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.dayofweek

df['hour_sin'] = np.sin(2*np.pi*df['hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['hour']/24)

df['dow_sin'] = np.sin(2*np.pi*df['dayofweek']/7)
df['dow_cos'] = np.cos(2*np.pi*df['dayofweek']/7)

df = df.dropna()

print("Feature 생성 완료!")


# ============================================================
# 3. 클러스터별 데이터 분리
# ============================================================
df_c0 = df[df["cluster"] == 0].copy()  # 0: 중/저혼잡형
df_c1 = df[df["cluster"] == 1].copy()  # 1: 상시혼잡형

print(f"Cluster 0 데이터 수: {len(df_c0)}")
print(f"Cluster 1 데이터 수: {len(df_c1)}")


# ============================================================
# 4. Train/Test Split (최근 30일 test)
# ============================================================
def split_data(df, test_days=30):
    split_date = df["date"].max() - pd.Timedelta(days=test_days)
    train = df[df["date"] <= split_date]
    test = df[df["date"] > split_date]

    drop_cols = ["traffic_volume", "intersection", "datetime", "date", "slot"]

    # drop_cols 제거
    drop_cols = [col for col in drop_cols if col in df.columns]

    X_train = train.drop(columns=drop_cols)
    y_train = train["traffic_volume"]
    X_test = test.drop(columns=drop_cols)
    y_test = test["traffic_volume"]

    return X_train, y_train, X_test, y_test



# ============================================================
# 5. 모델 학습 + 평가 함수
# ============================================================
def train_xgb(X_train, y_train):
    model = XGBRegressor(
        n_estimators=400,
        max_depth=10,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=1.0,
        gamma=1,
        min_child_weight=2,
        random_state=42
    )
    model.fit(X_train, y_train)
    return model


def evaluate(model, X_test, y_test):
    pred = model.predict(X_test)
    rmse = (mean_squared_error(y_test, pred))**0.5
    mae = mean_absolute_error(y_test, pred)
    return rmse, mae



# ============================================================
# 6. 클러스터별 모델 학습
# ============================================================

# Cluster 0 = 중/저혼잡형
X_train0, y_train0, X_test0, y_test0 = split_data(df_c0)
model0 = train_xgb(X_train0, y_train0)
rmse0, mae0 = evaluate(model0, X_test0, y_test0)

# Cluster 1 = 상시혼잡형
X_train1, y_train1, X_test1, y_test1 = split_data(df_c1)
model1 = train_xgb(X_train1, y_train1)
rmse1, mae1 = evaluate(model1, X_test1, y_test1)



# ============================================================
# 7. 결과 출력
# ============================================================
print("\n==============================")
print("📌 클러스터별 예측 성능")
print("==============================")

print(f"[Cluster 0 - 중/저혼잡형]\nRMSE: {rmse0:.2f}, MAE: {mae0:.2f}")
print(f"[Cluster 1 - 상시혼잡형]\nRMSE: {rmse1:.2f}, MAE: {mae1:.2f}")

print("\n🎉 전체 파이프라인 완료!")


데이터셋 구조:
             datetime intersection  traffic_volume  temp  precipitation  wind  \
0 2024-11-02 00:00:00         2공단3             341  15.2            0.0   1.1   
1 2024-11-02 01:00:00         2공단3             190  15.6            0.0   1.4   
2 2024-11-02 02:00:00         2공단3             207  14.1            0.0   0.7   
3 2024-11-02 03:00:00         2공단3             180  13.5            0.0   0.5   
4 2024-11-02 04:00:00         2공단3             230  13.2            0.0   0.0   

   is_offday       date  hour  dayofweek  month slot  is_rain  cluster  
0          1 2024-11-02     0          5     11   기타        0        0  
1          1 2024-11-02     1          5     11   기타        0        0  
2          1 2024-11-02     2          5     11   기타        0        0  
3          1 2024-11-02     3          5     11   기타        0        0  
4          1 2024-11-02     4          5     11   기타        0        0  
Feature 생성 완료!
Cluster 0 데이터 수: 345234
Cluster 1 데이터 수: 260430

📌 

### 클러스터링 그룹별로 교통량 예측 모델을 만들려고 했으나, 같은 그룹이라고 해도 사실상 개별 교차로별로 패턴이 달라 혼잡도 편중이 심한 현상이 발생함. 따라서, 실제 중요한 구간인 상시혼잡형에 집중해 모델을 정교화하기로 했음. 즉, 상시혼잡형 교차로별 개별 예측 모델을 만드는 것. -> 문제 있는 구간을 찾아 그 구간을 집중 개선

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt


# ============================================================
# 1. 데이터 불러오기 + 클러스터 정보 병합
# ============================================================
df = pd.read_csv("../data/merged/traffic_weather_features.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.floor('D')

# 클러스터링 정보
features = pd.read_csv("../data/merged/cluster_features.csv")
df = df.merge(features[['intersection', 'cluster']], on='intersection', how='left')

# 상시혼잡형만 선택 (cluster=1)
df_hot = df[df['cluster'] == 1].copy()

print(f"상시혼잡형 교차로 개수: {df_hot['intersection'].nunique()}")
print(df_hot['intersection'].unique())
print("\n데이터 확인:")
print(df_hot.head())


# ============================================================
# 2. Feature Engineering 함수 (교차로별 적용)
# ============================================================
def add_features(df_sub):
    df_sub = df_sub.sort_values("datetime").copy()

    # Lag features
    df_sub['lag1'] = df_sub['traffic_volume'].shift(1)
    df_sub['lag2'] = df_sub['traffic_volume'].shift(2)
    df_sub['lag3'] = df_sub['traffic_volume'].shift(3)
    df_sub['lag24'] = df_sub['traffic_volume'].shift(24)

    # Rolling
    df_sub['roll3'] = df_sub['traffic_volume'].rolling(3).mean()
    df_sub['roll6'] = df_sub['traffic_volume'].rolling(6).mean()
    df_sub['roll24'] = df_sub['traffic_volume'].rolling(24).mean()

    # Time features
    df_sub['hour'] = df_sub['datetime'].dt.hour
    df_sub['dayofweek'] = df_sub['datetime'].dt.dayofweek

    df_sub['hour_sin'] = np.sin(2*np.pi*df_sub['hour']/24)
    df_sub['hour_cos'] = np.cos(2*np.pi*df_sub['hour']/24)

    df_sub['dow_sin'] = np.sin(2*np.pi*df_sub['dayofweek']/7)
    df_sub['dow_cos'] = np.cos(2*np.pi*df_sub['dayofweek']/7)

    df_sub = df_sub.dropna()
    return df_sub


# ============================================================
# 3. Train/Test Split 함수
# ============================================================
def split_data(df_sub, test_days=30):
    split_date = df_sub['date'].max() - pd.Timedelta(days=test_days)
    train = df_sub[df_sub['date'] <= split_date]
    test = df_sub[df_sub['date'] > split_date]

    drop_cols = ["traffic_volume", "intersection", "datetime", "date", "slot"]
    drop_cols = [c for c in drop_cols if c in df_sub.columns]

    X_train = train.drop(columns=drop_cols)
    y_train = train["traffic_volume"]

    X_test = test.drop(columns=drop_cols)
    y_test = test["traffic_volume"]

    return X_train, y_train, X_test, y_test


# ============================================================
# 4. XGBoost 훈련 함수
# ============================================================
def train_xgb(X_train, y_train):
    model = XGBRegressor(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.07,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, y_train)
    return model


# ============================================================
# 5. 개별 교차로 모델 학습
# ============================================================
results = []

hot_intersections = df_hot['intersection'].unique()

for inter in hot_intersections:
    df_sub = df_hot[df_hot['intersection'] == inter].copy()
    df_sub = add_features(df_sub)

    if len(df_sub) < 200:
        print(f"❌ 데이터 적음 → 스킵: {inter}")
        continue

    X_train, y_train, X_test, y_test = split_data(df_sub)

    model = train_xgb(X_train, y_train)
    pred = model.predict(X_test)

    rmse = (mean_squared_error(y_test, pred))**0.5
    mae = mean_absolute_error(y_test, pred)

    results.append([inter, rmse, mae])

    print(f"✅ 완료: {inter} | RMSE={rmse:.2f}, MAE={mae:.2f}")


# ============================================================
# 6. 결과 정리
# ============================================================
results_df = pd.DataFrame(results, columns=["intersection", "RMSE", "MAE"])
results_df = results_df.sort_values("RMSE")

print("\n==============================")
print("📌 상시혼잡형 교차로별 예측 성능")
print("==============================")
print(results_df)


상시혼잡형 교차로 개수: 31
['고재4' '구성3' '굴울(변전소)4' '남부5' '도로원점3' '방아다리(이마트)4' '번영로(갤러리아)4' '불당4'
 '불당현대4' '삼성전관3' '새말(한라APT입구)4' '성정롯데4' '시청4' '쌍용도서관3' '쌍용동4' '쌍용초교4'
 '여성회관4' '용암4' '원형육교4' '이수(3산업)4' '인쇄창4' '일봉산4' '천안대로4' '청당동입구4' '청삼4'
 '하신3' '두정역사거리' '백석사거리' '백석충전소사거리' '서부대로사거리' '한들사거리']

데이터 확인:
              datetime intersection  traffic_volume  temp  precipitation  \
48 2024-11-02 00:00:00          고재4             492  15.2            0.0   
49 2024-11-02 01:00:00          고재4             359  15.6            0.0   
50 2024-11-02 02:00:00          고재4             351  14.1            0.0   
51 2024-11-02 03:00:00          고재4             284  13.5            0.0   
52 2024-11-02 04:00:00          고재4             378  13.2            0.0   

    wind  is_offday       date  hour  dayofweek  month slot  is_rain  cluster  
48   1.1          1 2024-11-02     0          5     11   기타        0        1  
49   1.4          1 2024-11-02     1          5     11   기타        0        1  
50   0.7    

### 그런데 상시혼잡형 교차로의 개별 예측 모델을 만드는 것에서도 문제가 생김. 개별 교차로 데이터를 train/test로 나누다보니 데이터가 부족 -> variance 폭증!! 모든 교차로가 완전히 독립된 모델이 됐기 때문. 또한 상시혼잡형 교차로는 변동성이 매우 커서 RMSE가 높게 나올 뿐더러 각 교차로마다 패턴 자체가 다를 수 있음(군집을 한 번 더 세분화해야 할 가능성)

1️⃣ 문제 정의

교차로별 시간대 교통량을 예측하는 모델을 만들고 있음.
→ 신호 최적화, 혼잡 관리에 활용하려는 목적.

2️⃣ 지금까지 한 일

날씨 + 시간 + 요일 + lag + rolling 등 기본 feature 생성

LR / RandomForest / XGBoost 비교
→ XGBoost가 가장 좋음

단, 상시혼잡 교차로는 변동성이 너무 커서 RMSE가 높게 나옴.

3️⃣ 클러스터링 적용

교차로별 평균/표준편차/피크/날씨반응 등으로 KMeans

2개 그룹으로 나뉨:

Cluster 0 = 중·저혼잡형

Cluster 1 = 상시혼잡형

4️⃣ 클러스터 기반 모델링

처음 시도: “클러스터별 1개 모델”
→ 성능 개선 거의 없음.

5️⃣ 현재 시도 중

**상시혼잡 교차로들(31곳)**만 따로
→ 교차로별 개별 XGBoost 모델 생성

결과: RMSE는 다소 크지만, 상시혼잡 특성상 자연스러운 수준
(기본 교통량 자체가 크고 변동폭이 커서 오차도 자연히 큼)

6️⃣ 지금 맞닥뜨린 핵심 문제

상시혼잡형은 변동이 심함

교차로별 데이터 수가 적어서 하나의 모델이 학습하기 어려움

현재 feature만으로는 각 교차로 고유 패턴을 충분히 표현하지 못함